In [1]:
!pip install mysql
!pip install mysql-connector-python


In [2]:
import mysql.connector
import getpass

def authenticate_manager():
    try:
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password="December@2004",
            database="team"
        )

        if mydb.is_connected():
            print("Connected to MySQL database")

            cursor = mydb.cursor()

            # Take user input for credentials
            username = input("Enter username: ")
            password = getpass.getpass("Enter password: ")

            query = "SELECT * FROM Manager WHERE username = %s AND password = %s"
            cursor.execute(query, (username, password))
            manager = cursor.fetchone()

            if manager:
                print("Login successful!")
                # Additional code after successful login
            else:
                print("Invalid credentials. Please check your username and password.")

            cursor.close()
            mydb.close()
        else:
            print("Connection to MySQL failed")

    except mysql.connector.Error as error:
        print("Error while connecting to MySQL", error)

authenticate_manager()

Connected to MySQL database
Enter username: Rujul@bcci
Enter password: ········
Login successful!


In [3]:
import mysql.connector
from tabulate import tabulate

def show_player_stats():
    try:
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password="December@2004",
            database="team"
        )

        if mydb.is_connected():
            print("Connected to MySQL database")

            cursor = mydb.cursor()

            query = "SELECT * FROM player_stats"
            cursor.execute(query)
            player_stats = cursor.fetchall()

            if player_stats:
                headers = [i[0] for i in cursor.description]  # Get column names
                print(tabulate(player_stats, headers=headers, tablefmt="grid"))
            else:
                print("No player stats found.")

            cursor.close()
            mydb.close()
        else:
            print("Connection to MySQL failed")

    except mysql.connector.Error as error:
        print("Error while connecting to MySQL", error)

In [4]:
import mysql.connector
from tabulate import tabulate

def display_players_table():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="December@2004",
        database="team"
    )

    if mydb.is_connected():
        print("Connected to MySQL database")

        cursor = mydb.cursor()
        select_query = "SELECT player_id, player_name, role FROM Players"
        cursor.execute(select_query)
        players = cursor.fetchall()

        headers = ["Player ID", "Player Name", "Role"]
        print("Players Table:")
        print(tabulate(players, headers=headers, tablefmt='pretty'))

        cursor.close()
        mydb.close()
    else:
        print("Connection to MySQL failed")

def schedule_match():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="December@2004",
        database="team"
    )

    if mydb.is_connected():
        print("Connected to MySQL database")

        display_players_table()

        match_id = input("Enter the match ID you want to schedule: ")

        players_required = {
            "Batsman": 7,   # 7 batsmen
            "Bowler": 4     # 4 bowlers
        }

        lineup = []
        cursor = mydb.cursor()
        for role, count in players_required.items():
            player_ids = []
            for i in range(count):
                player_id = input(f"Enter player ID for {role} {i+1}: ")
                player_ids.append(player_id)

            select_query = f"SELECT player_id, player_name FROM Players WHERE role = '{role}' AND player_id IN ({','.join(player_ids)})"
            cursor.execute(select_query)
            players = cursor.fetchall()
            lineup.extend(players)

        add_lineup_query = "INSERT INTO Match_Lineup (match_id, player_id) VALUES (%s, %s)"
        lineup_values = [(match_id, pid) for pid, _ in lineup]
        cursor.executemany(add_lineup_query, lineup_values)
        mydb.commit()

        headers = ["Player ID", "Player Name"]
        print("Selected Lineup:")
        print(tabulate(lineup, headers=headers, tablefmt='pretty'))

        cursor.close()
        mydb.close()
    else:
        print("Connection to MySQL failed")

In [5]:
import mysql.connector
from tabulate import tabulate

def display_coaches():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="December@2004",
        database="team"
    )

    if mydb.is_connected():
        cursor = mydb.cursor()
        select_query = select_query = "SELECT * FROM Coaches"

        cursor.execute(select_query)
        coaches = cursor.fetchall()

        headers = ["Coach ID", "Coach Name"]
        print("Coach Table:")
        print(tabulate(coaches, headers=headers, tablefmt='pretty'))

        cursor.close()
        mydb.close()
    else:
        print("Connection to MySQL failed")

def display_training_schedule():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="December@2004",
        database="team"
    )

    if mydb.is_connected():
        cursor = mydb.cursor()
        select_query = "SELECT * FROM Training_Schedule"
        cursor.execute(select_query)
        schedule = cursor.fetchall()

        headers = ["Schedule ID", "Details", "Notes", "Duration", "Equipment", "Coach ID", "Player ID"]
        print("Training Schedule:")
        print(tabulate(schedule, headers=headers, tablefmt='pretty'))

        cursor.close()
        mydb.close()
    else:
        print("Connection to MySQL failed")

def assign_training_schedule():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="December@2004",
        database="team"
    )

    if mydb.is_connected():
        display_coaches()

        display_training_schedule()

        schedule_details = input("Enter schedule details: ")
        notes = input("Enter notes: ")
        duration = int(input("Enter duration (in minutes): "))
        equipment_required = input("Enter equipment required: ")
        coach_id = input("Enter coach ID: ")
        player_id = input("Enter player ID: ")

        add_schedule_query = "INSERT INTO Training_Schedule (schedule_details, notes, duration, equipment_required, coach_id, player_id) VALUES (%s, %s, %s, %s, %s, %s)"
        schedule_values = (schedule_details, notes, duration, equipment_required, coach_id, player_id)

        cursor = mydb.cursor()
        cursor.execute(add_schedule_query, schedule_values)
        mydb.commit()

        display_training_schedule()

        cursor.close()
        mydb.close()
    else:
        print("Connection to MySQL failed")

In [6]:
import mysql.connector
from tabulate import tabulate

def display_players_by_strike_rate():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="December@2004",
        database="team"
    )

    if mydb.is_connected():
        cursor = mydb.cursor()
        query = """
        SELECT p.player_id, p.player_name, MAX(ps.strike_rate) as max_strike_rate
        FROM Players p
        JOIN player_stats ps ON p.player_id = ps.player_id
        WHERE p.role = 'Batsman'
        GROUP BY p.player_id
        ORDER BY max_strike_rate DESC
        """
        cursor.execute(query)
        players = cursor.fetchall()

        headers = ["Player ID", "Player Name", "Max Strike Rate"]
        print("Players Sorted by Max Strike Rate (Descending):")
        print(tabulate(players, headers=headers, tablefmt='pretty'))

        cursor.close()
        mydb.close()
    else:
        print("Connection to MySQL failed")

In [7]:
import mysql.connector
from tabulate import tabulate

def display_players_by_batting_avg():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="December@2004",
        database="team"
    )

    if mydb.is_connected():
        cursor = mydb.cursor()
        query = """
        SELECT p.player_id, p.player_name, MAX(ps.batting_avg) as max_batting_avg
        FROM Players p
        JOIN player_stats ps ON p.player_id = ps.player_id
        WHERE p.role = 'Batsman'
        GROUP BY p.player_id
        ORDER BY max_batting_avg DESC
        """
        cursor.execute(query)
        players = cursor.fetchall()

        headers = ["Player ID", "Player Name", "Max Batting Avg"]
        print("Players Sorted by Max Batting Average:")
        print(tabulate(players, headers=headers, tablefmt='pretty'))

        cursor.close()
        mydb.close()
    else:
        print("Connection to MySQL failed")

In [8]:
import mysql.connector
from tabulate import tabulate

def display_bowlers_by_economy():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="December@2004",
        database="team"
    )

    if mydb.is_connected():
        cursor = mydb.cursor()
        query = """
        SELECT p.player_id, p.player_name, MIN(ps.economy) as min_economy
        FROM Players p
        JOIN player_stats ps ON p.player_id = ps.player_id
        WHERE p.role = 'Bowler'
        GROUP BY p.player_id
        ORDER BY min_economy ASC
        """
        cursor.execute(query)
        bowlers = cursor.fetchall()

        headers = ["Player ID", "Player Name", "Min Economy"]
        print("Bowlers Sorted by Min Economy:")
        print(tabulate(bowlers, headers=headers, tablefmt='pretty'))

        cursor.close()
        mydb.close()
    else:
        print("Connection to MySQL failed")

In [14]:
print("Choose an action:")
print("1. Assign Training Schedule")
print("2. Display Players by Batting Average")
print("3. Display Players by Strike Rate")
print("4. Display Bowlers by Economy")
print("5. Schedule a match")
print("6. Display Player Stats")
choice = input("Enter your choice (1-6): ")

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="December@2004",
    database="team"
)

if mydb.is_connected():
    if choice == '1':
        assign_training_schedule()
    elif choice == '2':
        display_players_by_batting_avg()
    elif choice == '3':
        display_players_by_strike_rate()
    elif choice == '4':
        display_bowlers_by_economy()
    elif choice == '5':
        schedule_match()
    elif choice == '6':
        show_player_stats()
    else:
        print("Invalid choice. Please choose a number between 1 to 5.")

    mydb.close()
else:
    print("Connection to MySQL failed.")

Choose an action:
1. Assign Training Schedule
2. Display Players by Batting Average
3. Display Players by Strike Rate
4. Display Bowlers by Economy
5. Schedule a match
6. Display Player Stats
Enter your choice (1-6): 1
Coach Table:
+-----+---------------+----------+------------+
|     |               | Coach ID | Coach Name |
+-----+---------------+----------+------------+
| 121 |  John Smith   |   101    |     10     |
| 122 | Emma Johnson  |   103    |     8      |
| 123 | Alex Thompson |   102    |     12     |
| 124 | Olivia Brown  |   104    |     6      |
| 125 | Daniel Wilson |   105    |     15     |
+-----+---------------+----------+------------+
Training Schedule:
+-------------+------------------------+-------------------------------+----------+-----------------------+----------+-----------+
| Schedule ID |        Details         |             Notes             | Duration |       Equipment       | Coach ID | Player ID |
+-------------+------------------------+---------------